In [1]:
import pandas as pd

In [3]:
pip install apify-client

Note: you may need to restart the kernel to use updated packages.


In [7]:
input = ["sportscar","car"]

In [ ]:
from apify_client import ApifyClient
from dotenv import load_dotenv
import os

load_dotenv()

# Initialize the ApifyClient with your API token
apikey = os.getenv('API')
client = ApifyClient(apikey)

#Prepare the Actor input
run_input = {
    #"hashtags": ["fyp"],
    "searchQueries": input,
    "resultsPerPage": 50,
    "profileScrapeSections": ["videos"],
    "profileSorting": "latest",
    "excludePinnedPosts": False,
    "searchSection": "",
    "maxProfilesPerQuery": 10,
    "scrapeRelatedVideos": False,
    "shouldDownloadVideos": False,
    "shouldDownloadCovers": False,
    "shouldDownloadSubtitles": False,
    "shouldDownloadSlideshowImages": False,
    "shouldDownloadAvatars": False,
    "shouldDownloadMusicCovers": False,
    "proxyCountryCode": "None",
}

# Run the Actor and wait for it to finish
run = client.actor("GdWCkxBtKWOsKjdch").call(run_input=run_input)

# Fetch and print Actor results from the run's dataset (if there are any)
for item in client.dataset(run["defaultDatasetId"]).iterate_items():
    print(item)


[apify.tiktok-scraper runId:UH9QVdJ1OwT2wzQmH] -> Status: RUNNING, Message: 
[apify.tiktok-scraper runId:UH9QVdJ1OwT2wzQmH] -> 2025-07-30T07:02:30.236Z ACTOR: Pulling Docker image of build 7OshLXlRoiesBgRW8 from registry.
[apify.tiktok-scraper runId:UH9QVdJ1OwT2wzQmH] -> 2025-07-30T07:02:30.248Z ACTOR: Creating Docker container.
[apify.tiktok-scraper runId:UH9QVdJ1OwT2wzQmH] -> 2025-07-30T07:02:30.341Z ACTOR: Starting Docker container.
[apify.tiktok-scraper runId:UH9QVdJ1OwT2wzQmH] -> 2025-07-30T07:02:30.549Z Will run command: xvfb-run -a -s "-ac -screen 0 1920x1080x24+32 -nolisten tcp" /bin/sh -c ./start_xvfb_and_run_cmd.sh && npm run start:prod --silent
[apify.tiktok-scraper runId:UH9QVdJ1OwT2wzQmH] -> 2025-07-30T07:02:31.521Z INFO  System info {"apifyVersion":"3.4.2","apifyClientVersion":"2.12.6","crawleeVersion":"3.13.9","osType":"Linux","nodeVersion":"v20.19.4"}
[apify.tiktok-scraper runId:UH9QVdJ1OwT2wzQmH] -> 2025-07-30T07:02:31.724Z INFO  Starting the crawler.
[apify.tiktok-scr

{'id': '7294654036819709189', 'text': 'Prank scarying people with my Corvette C8!\n\n#cars #sportscar #luxury #supercars #prank #corvette #c8 #chevrolet', 'textLanguage': 'en', 'createTime': 1698938100, 'createTimeISO': '2023-11-02T15:15:00.000Z', 'isAd': False, 'authorMeta': {'id': '7238763041771750405', 'name': 'kaduzshop', 'profileUrl': 'https://www.tiktok.com/@kaduzshop', 'nickName': 'Kaduz Shop 💎', 'verified': False, 'signature': '🚚 Envios para todo 🇧🇷\n💲Cupom 10% OFF: PRIMEIRA10 🚀\n↙️ Acesse nossa loja ↙️', 'bioLink': None, 'originalAvatarUrl': 'https://p19-common-sign-va.tiktokcdn-us.com/tos-maliva-avt-0068/c5829ce7ad93f71a4d556fbb7be95ec2~tplv-tiktokx-cropcenter:720:720.jpeg?dr=9640&refresh_token=be83c613&x-expires=1754031600&x-signature=iGfcDfIKUNwvxKB0eSxMGlTuHfY%3D&t=4d5b0474&ps=13740610&shp=a5d48078&shcp=81f88b70&idc=useast5', 'avatar': 'https://p19-common-sign-va.tiktokcdn-us.com/tos-maliva-avt-0068/c5829ce7ad93f71a4d556fbb7be95ec2~tplv-tiktokx-cropcenter:720:720.jpeg?dr=9

In [ ]:
client.dataset(run["defaultDatasetId"])

In [ ]:
df = pd.read_csv('/content/dataset_tiktok_thaifood.csv')

In [ ]:
df.drop(columns=['authorMeta.avatar','authorMeta.privateAccount','authorMeta.ttSeller','authorMeta.bioLink'], inplace=True)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1349 entries, 0 to 1348
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   authorMeta.name       1349 non-null   object
 1   authorMeta.nickName   1349 non-null   object
 2   authorMeta.verified   1349 non-null   bool  
 3   authorMeta.signature  1234 non-null   object
 4   authorMeta.fans       1349 non-null   int64 
 5   authorMeta.video      1349 non-null   int64 
 6   authorMeta.id         1349 non-null   int64 
 7   text                  1273 non-null   object
dtypes: bool(1), int64(3), object(4)
memory usage: 75.2+ KB


In [ ]:
import re
def contains_thai_characters(text):
    if pd.isna(text):
        return False
    return bool(re.search(r'[\u0E00-\u0E7F]', str(text)))

df['has_thai'] = df['authorMeta.signature'].apply(contains_thai_characters)

df_filtered = df[df['has_thai']].copy()

df_filtered.drop(columns=['has_thai'], inplace=True)

print("\nDataFrame after removing rows where 'authorMeta.signature' has no Thai characters:")
print(df_filtered)


DataFrame after removing rows where 'authorMeta.signature' has no Thai characters:
         authorMeta.name   authorMeta.nickName  authorMeta.verified  \
0         praneatseafood    ร้านอาหารประณีต 🦀✨                False   
1     nourivebliss.ncafe  Nourive Bliss N.Cafe                False   
2          krua.pa.pleon          ครัวพาเพลิน🍳                False   
4         annhomecooking        Annhomecooking                False   
5                bee_nil            BB’s World                False   
...                  ...                   ...                  ...   
1344        arm_watunyou        อาร์มหรอยอีหลี                False   
1345        arm_watunyou        อาร์มหรอยอีหลี                False   
1346        arm_watunyou        อาร์มหรอยอีหลี                False   
1347        arm_watunyou        อาร์มหรอยอีหลี                False   
1348        arm_watunyou        อาร์มหรอยอีหลี                False   

                                   authorMeta.signature  author

In [ ]:
df_filtered

,authorMeta.name,authorMeta.nickName,authorMeta.verified,authorMeta.signature,authorMeta.fans,authorMeta.video,authorMeta.id,text
0,praneatseafood,ร้านอาหารประณีต 🦀✨,False,IG : praneatseafood \nร้านอาหารไทยประยุกต์ @พั...,2669,251,7241918823389365253,✨เมนูของทานเล่นเพื่อเรียกน้ำย่อย อร่อยแบบไทยแท...
1,nourivebliss.ncafe,Nourive Bliss N.Cafe,False,มัทฉะลาเต้ 3in1🍵\nสูตรคลีน💚 ไม่มีน้ำตาล✅ ไม่มี...,2984,271,7115265433398297605,6 อาหารไทย ที่คนฝรั่งชอบมาก😋 #อาหารไทย #foodti...
2,krua.pa.pleon,ครัวพาเพลิน🍳,False,👩‍🍳พื้นที่แบ่งปันสูตรความอร่อย🍳\n💼 ติดต่องาน L...,250600,284,7279733968840016901,ต้มข่าปีกไก่🍃 อร่อยกลมกล่อม😋\n🥣ส่วนผสม\n• ปีกก...
4,annhomecooking,Annhomecooking,False,ฝากติดตามด้วยนะคะ สั่งซื้อสินค้าที่ LINE OA: @...,114800,547,6832525909912847361,ยำขนมจีน ใส่อะไรไม่ต้องเยอะ แค่นี้ก็อร่อยแล้วว...
5,bee_nil,BB’s World,False,Life in Saudi Arabia 🇸🇦|แชร์ชีวิตมุมต่างๆในซาอ...,5414,73,6593228475020214273,ทำอาหารไทยไปให้เพื่อนสามีในซาอุ🇸🇦 ชิมครั้งแรก…...
...,...,...,...,...,...,...,...,...
1344,arm_watunyou,อาร์มหรอยอีหลี,False,page : หรอยอีหลี,970300,213,6957444182807397377,#ปลานิลผัดตะไคร้ #อาร์มหรอยอีหลี #หรอยอีหลี
1345,arm_watunyou,อาร์มหรอยอีหลี,False,page : หรอยอีหลี,970300,213,6957444182807397377,กินข้าวกับน้ำพริกเมนูโปรดของผมกับพี่เอิร์ธ #เอ...
1346,arm_watunyou,อาร์มหรอยอีหลี,False,page : หรอยอีหลี,970300,213,6957444182807397377,#ถั่วพูผัดน้ำมันหอย #อาร์มหรอยอีหลี #หรอยอีหลี
1347,arm_watunyou,อาร์มหรอยอีหลี,False,page : หรอยอีหลี,970300,213,6957444182807397377,#ผัดมะเขือพวงหมูสับ #อาร์มหรอยอีหลี #หรอยอีหลี


In [ ]:
dfsortbyfans = df_filtered.sort_values(by='authorMeta.fans',ascending=False)
dfsortbyfans

,authorMeta.name,authorMeta.nickName,authorMeta.verified,authorMeta.signature,authorMeta.fans,authorMeta.video,authorMeta.id,text
128,yeeyeehtay0113,เจ้าหญิงกบ❤️,False,🙏❤️ครอบครัวสายฮา💝🙏\n ❤️รับงานรีวิวค่ะ: LINE👉...,9000000,3034,6864526597882463233,อร่อย​มาก​ค่ะ​❤️
100,bella_bell22,เบลล่า วิถีไทย,False,ติดต่องาน Line ID : bellsy\n📍เล่นช่องนี้ช่องเด...,7200000,1314,6528496150231695362,โจ๊กมาม่า #เบลล่าวิถีไทย #นักชิมอาหาร #11x11Fo...
39,bella_bell22,เบลล่า วิถีไทย,False,ติดต่องาน Line ID : bellsy\n📍เล่นช่องนี้ช่องเด...,7200000,1314,6528496150231695362,ลองกินยาหนมกับนมจืดอร่อยมาก #เบลล่าวิถีไทย #ห้...
86,bella_bell22,เบลล่า วิถีไทย,False,ติดต่องาน Line ID : bellsy\n📍เล่นช่องนี้ช่องเด...,7200000,1314,6528496150231695362,กินมะม่วงคุ้ยแก่วแซ่บๆ #FoodFestOnTikTok #tikt...
110,bella_bell22,เบลล่า วิถีไทย,False,ติดต่องาน Line ID : bellsy\n📍เล่นช่องนี้ช่องเด...,7200000,1314,6528496150231695362,วุ้นกรอบ #เบลล่าวิถีไทย #tiktokพากิน #กินหนมกั...
...,...,...,...,...,...,...,...,...
300,zxcvbjyykuo,tiktokพากิน,False,แตกง่ายได้จริงต้องที่นี้\n📲 𝐋𝐈𝐍𝐄 𝐈𝐃 ➤ @NP69 (ม...,24,3,7324168435356845099,#กิน #แซลมอน #อร่อยบอกต่อ #วาซาบิ #tiktokพากิน
301,zxcvbjyykuo,tiktokพากิน,False,แตกง่ายได้จริงต้องที่นี้\n📲 𝐋𝐈𝐍𝐄 𝐈𝐃 ➤ @NP69 (ม...,24,3,7324168435356845099,#tiktokพากิน #กิน #ยํามะนาวกุ้งสด #ส้มแข่ว #อร่อย
302,zxcvbjyykuo,tiktokพากิน,False,แตกง่ายได้จริงต้องที่นี้\n📲 𝐋𝐈𝐍𝐄 𝐈𝐃 ➤ @NP69 (ม...,24,3,7324168435356845099,หอยนางรมสุราษฏร์ #tiktokพากิน #tiktokuni #อาหาร
479,timj0004,รวมดาวtiktokพากิน,False,ขออนุญาตทุกท่านนำคลิปมาเผยแพร่ ไม่ได้มีเจตนา\n...,15,2,7244908691156878341,ส้มตำรสเด็ดกุ้งสดแบบจัดจ้าน #ตำกุ้งสด #กิน #กิ...


In [ ]:
dfsortbyfansunique = dfsortbyfans.drop_duplicates(subset=['authorMeta.name'])

In [ ]:
dfsortbyfansunique.head(20)

,authorMeta.name,authorMeta.nickName,authorMeta.verified,authorMeta.signature,authorMeta.fans,authorMeta.video,authorMeta.id,text
128,yeeyeehtay0113,เจ้าหญิงกบ❤️,False,🙏❤️ครอบครัวสายฮา💝🙏\n ❤️รับงานรีวิวค่ะ: LINE👉...,9000000,3034,6864526597882463233,อร่อย​มาก​ค่ะ​❤️
100,bella_bell22,เบลล่า วิถีไทย,False,ติดต่องาน Line ID : bellsy\n📍เล่นช่องนี้ช่องเด...,7200000,1314,6528496150231695362,โจ๊กมาม่า #เบลล่าวิถีไทย #นักชิมอาหาร #11x11Fo...
54,kinkubky,กินกับกี้,False,Youtubeช่อง กินกับกี้\nติดต่องาน Line :@thelem...,5100000,2262,6809176177791632385,#ฉ่ํามั๊ยละ #กิน #ตํากุ้งสด #อร่อย #แซ่บ #tikt...
556,icezy159,ไอซ์ซี่ กินยั่ว,False,ติดต่องาน lineid : icezy168\nเพจ ไอซ์ซี่ กินยั...,5000000,1684,6519602653701783554,แกงเห็ดเผาะ #ยั่วๆจร้า #นักชิมอาหาร #อร่อยบอกต...
151,achasai,อาลี่ก๋ง. ( 4.6 M ) 🪞👸💗🌷✨,False,For work // line : @952eshvp\nInstagram: aach...,4600000,3424,75635744318,งู๊ยยยย ไข่ดองและแซลมอนดอง 🤤 อร่อยมากกก ฟิน 🫶🏻...
55,best_babeee,เบสเบบี๋ 👶🏻,False,FB: Apicha Insuwan \nIG : best_apicha \nติดต่อ...,3500000,4637,73975939911,ทานเล่นเบาๆกับแม่ยัยบี๋ งานวัดไร่ขิงปี68 🍨🧀🍗🍹🍫...
41,alumilightkitchen7,พ่อบ้านสดใสแม่บ้านหดหู่,False,ขอบคุณทุกคนที่ติดตาม🙏🥰\nID:suningdameinv\nเพจ:...,3200000,1157,6564513206260334593,เมื่อทายาทอยากกินต้องได้กิน #ครัวอลูมีไล้ #พ่อ...
887,tiktokshop_th,TikTokShop_TH,True,TikTok Shop ช้อปสนุกทุกตะกร้า🛒,3100000,1127,7095929463933912070,เมื่อคุณลืมพกความมั่นใจมาจากบ้าน บู้สความมั่น...
95,mastertomtomxoxo,MasterTomTom,False,🎩\nwork- @mastertomtom (มี@)\nงดนำคลิปไปใช้ต่อ...,3100000,274,7014785101205423130,ทำ‘ส้มตำยายเพลิงนรกร้านดัง กรอบแซ่บระเบิดล้นคร...
99,hector.book,แล้วแต่แฟนเลยครับ,False,Book & Air แฟนผมกินเก่ง 😂\nเจ้าของแบรนด์ lumiskin,3000000,424,78239935037,ตอบกลับ @dm.fs คุณแม่แฟนยึดช่องผมแล้วครับ เก่ง...


In [ ]:
pip install pythainlp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 82.8 MB/s eta 0:00:00


In [ ]:
from pythainlp.tokenize import word_tokenize
from pythainlp.corpus import thai_stopwords

# Load Thai stopwords once
thai_stop_words = set(thai_stopwords())

def preprocess_text(text):
    if not isinstance(text, str): # Handle non-string inputs like None
        return ""

    emoji_pattern = re.compile(
        "["
        "\U0001F600-\U0001F64F"  # emoticons
        "\U0001F300-\U0001F5FF"  # symbols & pictographs
        "\U0001F680-\U0001F6FF"  # transport & map symbols
        "\U0001F1E0-\U0001F1FF"  # flags (iOS)
        "\U00002702-\U000027B0"  # Dingbats
        "\U000024C2-\U0001F251"
        "\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
        "\U00002600-\U000026FF"  # Miscellaneous Symbols
        "\U00002500-\U000025FF"  # Box Drawing, Block Elements
        "]+", flags=re.UNICODE
    )
    text = emoji_pattern.sub(r'', text)
    # Remove characters that are NOT Thai letters, numbers, #, @, or spaces
    text = re.sub(r'[^\u0E00-\u0E7F\s]', '', text)

    tokens = word_tokenize(text, keep_whitespace=False)


    tokens = [word for word in tokens if word not in thai_stop_words and word.strip() != '']

    return " ".join(tokens)


dfsortbyfansunique['processed_signature'] = dfsortbyfansunique['authorMeta.signature'].apply(preprocess_text)
dfsortbyfansunique['processed_text'] = dfsortbyfansunique['text'].apply(preprocess_text)

dfsortbyfansunique['influencer_combined_text'] = dfsortbyfansunique['processed_signature'] + " " + dfsortbyfansunique['processed_text']

brand_description_text = (
    "โค้ก กินกับอะไรก็อร่อย อาหารเผ็ดๆยิ่งต้องกินกับโค้กเลย "
)

processed_brand_text = preprocess_text(brand_description_text)

print("Processed Brand Text:")
print(processed_brand_text)
print("\nProcessed Influencer Texts (first 2 rows):")
print(dfsortbyfansunique[['authorMeta.name', 'influencer_combined_text']].head(2).to_string())

Processed Brand Text:
โค้ก กิน อร่อย อาหาร เผ็ด กิน โค้ก

Processed Influencer Texts (first 2 rows):
    authorMeta.name                                                                                  influencer_combined_text
128  yeeyeehtay0113                                                                        ครอบครัว สาย ฮา รับงาน รีวิว อร่อย
100    bella_bell22  ติดต่อ งาน เล่น ช่อง ช่อง ห้าม คลิป แจ้ง โจ๊ก มาม่า เบล ล่า วิถี ไทย ชิม อาหาร โรบินฮู้ด กิน เชฟ มือใหม่


/tmp/ipython-input-16-271021785.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfsortbyfansunique['processed_signature'] = dfsortbyfansunique['authorMeta.signature'].apply(preprocess_text)
/tmp/ipython-input-16-271021785.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfsortbyfansunique['processed_text'] = dfsortbyfansunique['text'].apply(preprocess_text)
/tmp/ipython-input-16-271021785.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .l

In [ ]:
print(dfsortbyfansunique['influencer_combined_text'].head())
print("\n"+processed_brand_text)

128                   ครอบครัว สาย ฮา รับงาน รีวิว อร่อย
100    ติดต่อ งาน เล่น ช่อง ช่อง ห้าม คลิป แจ้ง โจ๊ก ...
54     ช่อง กิน กี้ ติดต่อ งาน ยีน ฉ่ํามั๊ย กิน ตํา ก...
556    ติดต่อ งาน เพจ ไอซ์ ซี่ กิน ยั่ว แฟน เพจ เฟส ส...
151    อนุญาต ดูด คลิป กรณี งู๊ยยยย ไข่ ดอง แซลมอน ดอ...
Name: influencer_combined_text, dtype: object

โค้ก กิน อร่อย อาหาร เผ็ด กิน โค้ก


In [40]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Create a corpus of all texts (brand + all influencer texts)
corpus = [processed_brand_text] + dfsortbyfansunique['influencer_combined_text'].tolist()

# max_features can limit the vocabulary size to improve performance/focus on most important words
vectorizer = TfidfVectorizer(max_features=4000)

# Fit the vectorizer on the entire corpus and transform
tfidf_matrix = vectorizer.fit_transform(corpus)

# Separate the brand vector from influencer vectors
brand_vector = tfidf_matrix[0:1]
influencer_vectors = tfidf_matrix[1:]


In [ ]:
similarity_scores = cosine_similarity(brand_vector, influencer_vectors).flatten()

dfsortbyfansunique['relevance_score'] = similarity_scores

print("\nInfluencers with Relevance Scores:")
print(dfsortbyfansunique[['authorMeta.name', 'authorMeta.fans', 'relevance_score']].sort_values(by='relevance_score', ascending=False))


Influencers with Relevance Scores:
               authorMeta.name  authorMeta.fans  relevance_score
415             nurse.enjoyeat           324500         0.273034
188                  shibacook          2100000         0.250146
412                    j100.kg          1100000         0.247859
390                   faymily_            89200         0.236992
433   nutthanon_loetphatphicha          2600000         0.177942
...                        ...              ...              ...
959     ruennoppagaorestaurant              792         0.000000
339                  tiltok_90              211         0.000000
521                cainnnn_nnn              712         0.000000
1106           tiktok_ajarn.wi              177         0.000000
479                   timj0004               15         0.000000

[171 rows x 3 columns]


/tmp/ipython-input-37-470239746.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfsortbyfansunique['relevance_score'] = similarity_scores


In [42]:
weight_relevance = 0.6
weight_fans = 0.1

# Calculate the total score
max_fans = dfsortbyfansunique['authorMeta.fans'].max()
dfsortbyfansunique['normalized_fans'] = dfsortbyfansunique['authorMeta.fans'] / max_fans # Scale fans to 0-1

dfsortbyfansunique['total_score'] = (dfsortbyfansunique['relevance_score'] * weight_relevance) + \
                                (dfsortbyfansunique['normalized_fans'] * weight_fans)

# Sort by the total score to get your final ranked list
final_ranked_influencers = dfsortbyfansunique.sort_values(by='total_score', ascending=False)

print("\nFinal Ranked Influencers (by Total Score):")
print(final_ranked_influencers[['authorMeta.name', 'authorMeta.fans', 'relevance_score', 'total_score']].head(10))


Final Ranked Influencers (by Total Score):
              authorMeta.name  authorMeta.fans  relevance_score  total_score
188                 shibacook          2100000         0.250146     0.173421
415            nurse.enjoyeat           324500         0.273034     0.167426
412                   j100.kg          1100000         0.247859     0.160938
390                  faymily_            89200         0.236992     0.143186
433  nutthanon_loetphatphicha          2600000         0.177942     0.135654
128            yeeyeehtay0113          9000000         0.048802     0.129281
121                 mawin_twp          2900000         0.111934     0.099383
142              sukanyapakin          1000000         0.145463     0.098389
124              dingdongeiei           246300         0.157419     0.097188
54                   kinkubky          5100000         0.067425     0.097121


/tmp/ipython-input-42-922140075.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfsortbyfansunique['normalized_fans'] = dfsortbyfansunique['authorMeta.fans'] / max_fans # Scale fans to 0-1
/tmp/ipython-input-42-922140075.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfsortbyfansunique['total_score'] = (dfsortbyfansunique['relevance_score'] * weight_relevance) + \
